# Advent of Code - Day 5: Supply Stacks

In [1]:
import numpy as np
import pandas as pd

## Question 1 

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked crates, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a giant cargo crane capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her which crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates and the rearrangement procedure (your puzzle input). For example:

1 N Z\
2 M C D\
3 P 

move 1 from 2 to 1\
move 3 from 1 to 3\
move 2 from 2 to 1\
move 1 from 1 to 2\
In this example, there are three stacks of crates. Stack 1 contains two crates: crate Z is on the bottom, and crate N is on top. Stack 2 contains three crates; from bottom to top, they are crates M, C, and D. Finally, stack 3 contains a single crate, P.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:


 1 Z N D\
 2 M C\
 3 P
 
 
In the second step, three crates are moved from stack 1 to stack 3. Crates are moved one at a time, so the first crate to be moved (D) ends up below the second and third crates:

 1\
 2 M C\
 3 P D N Z
 
 
Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved one at a time, crate C ends up below crate M:

 
1 C M\
2\
3 P D N Z 

 
Finally, one crate is moved from stack 1 to stack 2:
 
 1 C\
 2 M\
 3 P D N Z
 
 
The Elves just need to know which crate will end up on top of each stack; in this example, the top crates are C in stack 1, M in stack 2, and Z in stack 3, so you should combine these together and give the Elves the message CMZ.

After the rearrangement procedure completes, what crate ends up on top of each stack?


Starting Arrangement:
 
 1 G F V H P S\
 2 G J F B V D Z M\
 3 G M L J N\
 4 N G Z V D W P\
 5 V R C B\
 6 V R S M P W L Z\
 7 T H P\
 8 Q R S N C H Z V
 9 F L G P V Q J 
 
 

## Discussion
Reading the text file into a pandas DataFrame will allow for easier manipulation to put the distinct inputs (number of blocks, start stack and end stack) into their own columns. This will allow for much easier looping. Using whitespace as a delimiter ensured the data was read into 6 columns, three with the words and three with integers. This meant I could easily drop the words column and the integers were already in the correct datatype. These columns were renamed for readability in the function. 

In [176]:
# read in CSV file to a pandas DataFrame (using whitespace as a delimiter)
moves = pd.read_csv('data/stackingmoves.txt', header=None, delim_whitespace=True)
# drop the columns with the words ('move', 'from', 'to'), so that only the integer columns remain
moves = moves.drop(columns=[0, 2, 4])
# rename colums for easier use within the function
moves = moves.rename(columns={1:'num', 3:'start', 5:'end'})

Using a 2D array for the stacks meant I could easily move parts of lists to other lists and loop over the different stacks in the array

In [170]:
# create a 2D array for the current state of the stacks
piles = [['G', 'F', 'V', 'H', 'P', 'S'],
['G', 'J', 'F', 'B', 'V', 'D', 'Z', 'M'],
['G', 'M', 'L', 'J', 'N'],
['N', 'G', 'Z', 'V', 'D', 'W', 'P'],
['V', 'R', 'C', 'B'],
['V', 'R', 'S', 'M', 'P', 'W', 'L', 'Z'],
['T', 'H', 'P'],
['Q', 'R', 'S', 'N', 'C', 'H', 'Z', 'V'],
['F', 'L', 'G', 'P', 'V', 'Q', 'J']]

In [179]:
def CrateMover(piles=piles, moves=moves, machine_number='9000'):
    """
    Moves crates from stack to stack, based on passed in instructions. 
    
    Parameters:
    -----------
    piles: 2D Array 
                A matrix holding the stacks and the contents of the stacks.
                
    moves: Pandas DataFrame
                A dataframe containing the moves needed to be made 
    
    machine_number: string
                A string of the machine number, which designates the stacking
                    order
    
    Returns 
    -------
    The string created by concatenating the top element of each stack 
    
    """
    
    
    # create a copy of the stacks, so that the original array is not changed
    stacks = [pile[:] for pile in piles]
    
    for i in range(len(moves)):
        # the number of items to move
        number = moves.loc[i, 'num']
        
        # the start stack number, reindexed
        start = moves.loc[i, 'start'] - 1
        # the end stack number, reindexed
        end = moves.loc[i, 'end'] - 1 
        
        if machine_number == '9000':
            # the items to move
            to_move = list(reversed(stacks[start][-number:]))
            
        elif machine_number == '9001':
            to_move = stacks[start][-number:]

        # add items to end (top) of new stack
        stacks[end].extend(to_move)
        
        # remove items from end (top) of old stack
        stacks[start] = stacks[start][:-number]
        
        
    return ''.join([stack[-1] if len(stack) > 0 else '' for stack in stacks])

In [173]:
CrateMover()

'FCVRLMVQP'

## Question 2 

As you watch the crane operator expertly rearrange the crates, you notice the process isn't following your prediction.

Some mud was covering the writing on the side of the crane, and you quickly wipe it away. The crane isn't a CrateMover 9000 - it's a CrateMover 9001.

The CrateMover 9001 is notable for many new and exciting features: air conditioning, leather seats, an extra cup holder, and the ability to pick up and move multiple crates at once.

Again considering the example above, the crates begin in the same configuration:

1 Z N\
2 M C D\
3 P 

Moving a single crate from stack 2 to stack 1 behaves the same as before:

1 Z N D\
2 M C\
3 P 

 
However, the action of moving three crates from stack 1 to stack 3 means that those three moved crates stay in the same order, resulting in this new configuration:

 
1\
2 M C\
3 P Z N D 
 
 
Next, as both crates are moved from stack 2 to stack 1, they retain their order as well:

1 M C\
2\
3 P Z N D 

Finally, a single crate is still moved from stack 1 to stack 2, but now it's crate C that gets moved:

1 M\
2 C\
3 P Z N D 

In this example, the CrateMover 9001 has put the crates in a totally different order: MCD.

Before the rearrangement process finishes, update your simulation so that the Elves know where they should stand to be ready to unload the final supplies. After the rearrangement procedure completes, what crate ends up on top of each stack?


### Discussion 
To answer this question I made a simple change to the stacking function above. Adding a machine number parameter allowed us to control, based on the input, which order of stacking should be used. For the CrateMover 9001 no list reversing was needed, so I only added the crates (sub-list) to the end of the new stack

In [175]:
CrateMover(piles, machine_number='9001')

'RWLWGJGFD'